In [1]:
import pandas as pd
import dask.dataframe as dd
from glob import glob

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

/opt/miniconda3/envs/gtu_eval_parallel/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63876 instead
  warnings.warn(
distributed.diskutils - INFO - Found stale lock file and directory '/Users/sean/projects/DS/bv_mapping/dask-worker-space/worker-w4ygcthj', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/sean/projects/DS/bv_mapping/dask-worker-space/worker-c_2elycg', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/sean/projects/DS/bv_mapping/dask-worker-space/worker-an6crfpc', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/sean/projects/DS/bv_mapping/dask-worker-space/worker-r1kst6a5', purging


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:63876/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:63876/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:63877,Workers: 4
Dashboard: http://127.0.0.1:63876/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:63896,Total threads: 2
Dashboard: http://127.0.0.1:63897/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:63882,


In [ ]:
zipcode_region = 0

In [3]:
df = dd.read_csv("./data/*.csv").compute()

In [4]:
df_shape = pd.read_csv(f"./zipcodes/zipcodes_shape_{zipcode_region}.csv")

In [5]:
df = df.join(df_shape.set_index("gid"), on="gid")
df = df.dropna()
df = df.set_index("gid")
df.to_parquet(f"joined_{zipcode_region}")

# {
#   "type": "Feature",
#   "geometry": {
#     "type": "Point",
#     "coordinates": [125.6, 10.1]
#   },
#   "properties": {
#     "name": "Dinagat Islands"
#   }
# }

In [6]:
df

,zcta5ce10,avg,score_1,score_2,score_3,score_4,score_5,st_asgeojson
gid,,,,,,,,
10029,21634,88.909739,7,31,41,151,191,"{""type"":""MultiPolygon"",""coordinates"":[[[[-76.2..."
10030,21635,88.026391,29,98,136,634,505,"{""type"":""MultiPolygon"",""coordinates"":[[[[-75.9..."
10032,21636,87.753325,10,79,107,317,314,"{""type"":""MultiPolygon"",""coordinates"":[[[[-75.8..."
10033,21638,87.515762,50,172,314,935,813,"{""type"":""MultiPolygon"",""coordinates"":[[[[-76.2..."
10034,21639,88.398649,35,147,268,1057,861,"{""type"":""MultiPolygon"",""coordinates"":[[[[-75.8..."
...,...,...,...,...,...,...,...,...
9935,29626,88.525615,186,407,827,2816,3045,"{""type"":""MultiPolygon"",""coordinates"":[[[[-82.8..."
9936,29627,88.137687,252,645,1408,4668,4190,"{""type"":""MultiPolygon"",""coordinates"":[[[[-82.6..."
9937,29628,87.492044,51,109,211,657,606,"{""type"":""MultiPolygon"",""coordinates"":[[[[-82.5..."


In [7]:
def insert_data_into_json(row):
    new_geojson = {}
    new_geojson["type"] = "Feature"
    new_geojson["geometry"] = eval(row["st_asgeojson"])
    
    tem = {}
    tem['zipcode'] = row["zcta5ce10"]
    for score_val in range(1,6):
        tem[f"score_{score_val}"] = row[f"score_{score_val}"]

    
    new_geojson["properties"] = tem
    
    row["st_asgeojson"] = new_geojson
    
    return row

In [8]:
df = pd.read_parquet(f"joined_{zipcode_region}")
df = df.apply(insert_data_into_json,axis=1)
df.to_parquet("converted")

In [22]:
import pandas as pd

In [23]:
df = pd.read_parquet(f"converted_{zipcode_region}")
df['st_asgeojson'].to_json(f"zipcode_region_{zipcode_region}.json",orient="values")

In [24]:
prefix = '{"type": "FeatureCollection", "features": '
suffix = '}'

In [25]:
with open(f"zipcode_region_{zipcode_region}.json",'r') as f:
    read_data = f.read()
with open(f"zipcode_region_{zipcode_region}.json",'w') as f:
    write_data = prefix+read_data+suffix
    f.write(write_data)

[{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-76.26789,38.343848],[-76.26451,38.346354],[-76.262931,38.343936],[-76.262672,38.343811],[-76.262428,38.343635],[-76.262313,38.343505],[-76.262062,38.343254],[-76.261802,38.343017],[-76.261543,38.342754],[-76.261337,38.342567],[-76.261169,38.342418],[-76.26107,38.3423],[-76.260864,38.34217],[-76.260704,38.342056],[-76.260467,38
